In [ ]:
project = 'saga-nvdb-prod-vlmh'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen henter ut lengden på det nåværende vegnettet for riks- og europaveg i Norge.

In [ ]:
query = f"""
SELECT 
  SUM(ST_LENGTH(geometri))/1000 vegnettKm
FROM 
  `{project}.standardized.veglenker`
WHERE
  # Nåværende vegnett
  (metadata.sluttdato IS NULL OR metadata.sluttdato >= CURRENT_DATE())

  # Hovedtrase
  AND type = "HOVED"

  # Europaveg og Riksveg
  AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

  # Eksisterende veg som er del av det operative vegnettet
  AND vegsystemreferanse.vegsystem.fase in ("V")

  # Behold bare ett løp for adskilte tunelløp
  AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

  AND typeVeg IN (
    "Rundkjøring",
    "Enkel bilveg",
    "Kanalisert veg",
    "Rampe");
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen henter ut det nåværende vegnettet for riks- og europaveg i Trøndelag fylke.

In [ ]:
query = f"""
SELECT 
  vegsystemreferanse.kortform, geometri
FROM 
  `{project}.standardized.veglenker`
WHERE
  # Trøndelag
  fylke = 50

  # Nåværende vegnett
  AND (metadata.sluttdato IS NULL OR metadata.sluttdato >= CURRENT_DATE())

  # Hovedtrase
  AND type = "HOVED"

  # Europaveg og Riksveg
  AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

  # Eksisterende veg som er del av det operative vegnettet
  AND vegsystemreferanse.vegsystem.fase in ("V")

  # Behold bare ett løp for adskilte tunelløp
  AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

  AND typeVeg IN (
    "Rundkjøring",
    "Enkel bilveg",
    "Kanalisert veg",
    "Rampe");
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen viser hvordan riks- og europavegnettet for et gitt tidspunkt kan hentes ut, med ellers samme filter som over.

In [ ]:
query = f"""
SELECT 
  vegsystemreferanse.kortform,
  geometri
FROM 
  `{project}.standardized.veglenker`
WHERE
  # Trøndelag
  fylke = 50

  # Vegnett for et gitt tidspunkt
  AND (
      # Opprettet før det aktuelle tidspunktet 
      metadata.startdato < '2020-01-01'
      AND
      # Avsluttet etter tidspunktet eller er fortsatt aktiv
      (metadata.sluttdato >= '2020-01-01' OR metadata.sluttdato IS NULL)
  ) 

  # Hovedtrase
  AND type = "HOVED"

  # Europaveg og Riksveg
  AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

  # Eksisterende veg som er del av det operative vegnettet
  AND vegsystemreferanse.vegsystem.fase in ("V")

  # Behold bare ett løp for adskilte tunelløp
  AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

  AND typeVeg IN (
    "Rundkjøring",
    "Enkel bilveg",
    "Kanalisert veg",
    "Rampe");
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen viser hvordan vegeier og vegens navn kan utledes fra de andre feltene.

In [ ]:
query = f"""
SELECT
  FORMAT("%s%s%d", 
    vegsystemreferanse.vegsystem.vegkategori, 
    vegsystemreferanse.vegsystem.fase, 
    vegsystemreferanse.vegsystem.nummer) vegnavn,
  IF(
    EXISTS(SELECT * FROM UNNEST(kontraktsomraader) WHERE REGEXP_CONTAINS(navn, "NV_")),
    "Stat, Nye Veier", 
    "Stat, Statens vegvesen") vegeier,
FROM 
  `{project}.standardized.veglenker`
WHERE
  # Trøndelag
  fylke = 50

  # Nåværende vegnett
  AND (metadata.sluttdato IS NULL OR metadata.sluttdato >= CURRENT_DATE())

  # Hovedtrase
  AND type = "HOVED"

  # Europaveg og Riksveg
  AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

  # Eksisterende veg som er del av det operative vegnettet
  AND vegsystemreferanse.vegsystem.fase in ("V")

  # Behold bare ett løp for adskilte tunelløp
  AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

  AND typeVeg IN (
    "Rundkjøring",
    "Enkel bilveg",
    "Kanalisert veg",
    "Rampe");
"""

print(query)

client.query(query).to_dataframe()


Denne spørringen viser hvordan man kan avdekke veglenkesekvenser som ble blitt opprettet i perioden 2020-01-01 til 2021-01-01. Merk at spørringen returnerer alle veglenker i sekvensen, ikke bare de som ble opprettet i den aktuelle perioden.

In [ ]:
query = f"""
WITH 
veglenker AS 
  (SELECT *
  FROM `{project}.standardized.veglenker`
  WHERE 
    # Hovedtrase
    type = "HOVED"

    # Europaveg og Riksveg
    AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

    # Eksisterende veg som er del av det operative vegnettet
    AND vegsystemreferanse.vegsystem.fase in ("V")

    # Behold bare ett løp for adskilte tunelløp
    AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

    AND typeVeg IN (
      "Rundkjøring",
      "Enkel bilveg",
      "Kanalisert veg",
      "Rampe")),

nye_veglenkesekvenser AS
  (SELECT
    veglenkesekvensid,
    ST_UNION_AGG(geometri) geometri,
    MIN(metadata.startdato) AS tidligste_startdato
  FROM veglenker
  GROUP BY veglenkesekvensid
  HAVING
   # Sekvensen ble opprettet i perioden
   tidligste_startdato BETWEEN '2020-01-01' AND '2021-01-01')

SELECT * FROM nye_veglenkesekvenser
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen viser hvordan man kan avdekke veglenkesekvenser som ble endelig avsluttet i perioden 2020-01-01 til 2021-01-01. Dette er en grei måte å få oversikt over større endringer i vegnettet, men den får ikke med seg de tilfeller hvor det blir avsluttet noen, men ikke alle, veglenker i en sekvens. [Se denne presentasjonen for mer informasjon om forholdet mellom veglenker og veglenkesekvenser.](https://vegvesen-my.sharepoint.com/:p:/g/personal/jan_kristian_jensen_vegvesen_no/ERTPmjdinZxIh23EwpQeigYBn_lFAzyJHuuisQyU4_qVhg).

Merk at spørringen returnerer alle veglenker i sekvensen, ikke bare de som ble avsluttet i den aktuelle perioden.

In [ ]:

query = f"""
WITH 
veglenker AS 
  (SELECT *
  FROM `{project}.standardized.veglenker`
  WHERE 
    # Hovedtrase
    type = "HOVED"

    # Europaveg og Riksveg
    AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

    # Eksisterende veg som er del av det operative vegnettet
    AND vegsystemreferanse.vegsystem.fase in ("V")

    # Behold bare ett løp for adskilte tunelløp
    AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

    AND typeVeg IN (
      "Rundkjøring",
      "Enkel bilveg",
      "Kanalisert veg",
      "Rampe")),

stengte_veglenkesekvenser AS
  (SELECT
    veglenkesekvensid,
    ST_UNION_AGG(geometri) geometri,
    COUNT(*) AS totalt_antall_veglenker,
    SUM(
      CASE
        WHEN metadata.sluttdato IS NOT NULL THEN 1
        ELSE 0
      END) antall_stengte_veglenker,
    MAX(metadata.sluttdato) AS siste_sluttdato
  FROM veglenker
  GROUP BY veglenkesekvensid
  HAVING
   # alle veglenkene i veglenkesekvensen er stengt
   totalt_antall_veglenker = antall_stengte_veglenker
   # siste sluttdato i perioden
   AND siste_sluttdato BETWEEN '2020-01-01' AND '2021-01-01')

SELECT * FROM stengte_veglenkesekvenser
"""

print(query)

client.query(query).to_dataframe()